In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import pandas as pd
from tqdm import tqdm, trange

In [43]:
df = pd.read_hdf('/work/jp464/striatum-sequence/output/performance.h5', 'data')
col = str(.3) + '-' + str(.1) + '-' + str(.1) + '-' + str(.1)
if col in df:
    df.at[0, col] += 1 
    df.at[1, col] = 10
else:
    df[col] = pd.Series([0,0])
    df.at[0, col], df.at[1, col] = 1, 10
print(df)
df.to_hdf('/work/jp464/striatum-sequence/output/performance.h5', 'data')

   0.2-0.1-0.1-0.1  0.3-0.1-0.1-0.1
0                3                1
1               10               10


In [71]:
df = pd.read_hdf('/work/jp464/striatum-sequence/output/performance.h5', 'data')
df

,70-0.05-1500-0.6
0,5
1,5959
2,2942
